<a href="https://colab.research.google.com/github/davanstrien/people-iiif/blob/main/03_download_loc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install httpx loguru fastcore sparqlwrapper 
!pip install tqdm --upgrade

     |████████████████████████████████| 81kB 3.9MB/s 
     |████████████████████████████████| 61kB 6.2MB/s 
     |████████████████████████████████| 61kB 6.1MB/s 
     |████████████████████████████████| 61kB 6.5MB/s 
     |████████████████████████████████| 235kB 33.2MB/s 
     |████████████████████████████████| 61kB 6.5MB/s 
     |████████████████████████████████| 51kB 4.6MB/s 
     |████████████████████████████████| 81kB 4.0MB/s 
  Found existing installation: tqdm 4.41.1
    Uninstalling tqdm-4.41.1:
      Successfully uninstalled tqdm-4.41.1


In [2]:
import pandas as pd
import httpx
import io
from pathlib import Path
from fastcore.all import *
from PIL import Image
import hashlib
from loguru import logger

In [3]:
from tqdm.notebook import tqdm
tqdm.pandas()

In [4]:
path = Path('/content/drive/MyDrive/IIIF_AI_project')

In [5]:
df = pd.read_csv("https://raw.githubusercontent.com/davanstrien/people-iiif/main/loc_harvester/image_by_subject.csv?token=ACEUI5MCABVMBM6BYRBWTQTAU7GAI", index_col=0)

In [6]:
df

,id,desc,term,source,url
0,tgm012273,"Library [of Congress, main reading room, Washi...",Glass,https://www.loc.gov/pictures/item/2016819413/,https://tile.loc.gov/storage-services/service/...
1,tgm012273,[Stained glass windows],Glass,https://www.loc.gov/pictures/item/2016819457/,https://tile.loc.gov/storage-services/service/...
2,tgm012273,Cone-shaped exterior feature at the Museum of ...,Glass,https://www.loc.gov/pictures/item/2018699148/,https://tile.loc.gov/storage-services/service/...
3,tgm012273,Cone-shaped exterior feature at the Museum of ...,Glass,https://www.loc.gov/pictures/item/2018699154/,https://tile.loc.gov/storage-services/service/...
4,tgm012273,"[Library of Congress (Washington, D.C.). Leade...",Glass,https://www.loc.gov/pictures/item/2001695198/,https://tile.loc.gov/storage-services/service/...
...,...,...,...,...,...
72702,tgm005206,Things we see at the National Academy of Design,Hugging,https://www.loc.gov/pictures/item/2011660524/,https://tile.loc.gov/storage-services/service/...
72703,tgm005206,"""Woman are too sentimental for jury duty"" --An...",Hugging,https://www.loc.gov/pictures/item/2011660530/,https://tile.loc.gov/storage-services/service/...
72704,tgm005206,"""I'll take care of you, Grandma!"" (after the o...",Hugging,https://www.loc.gov/pictures/item/2012648687/,https://tile.loc.gov/storage-services/service/...
72705,tgm005206,"Wed! ""Does oo love oo Tigy-Wigy?""",Hugging,https://www.loc.gov/pictures/item/2012648780/,https://tile.loc.gov/storage-services/service/...


In [7]:
df['urlhash'] = pd.util.hash_pandas_object(df.url, index=False)

In [8]:
Path(path/'loc-images-labels').mkdir(exist_ok=True)
save_path = Path(path/'loc-images-labels')

In [9]:
def download_image(row, path=save_path):
    url = row.url
    hash = row.urlhash
    ext = url.split('.')[-1]
    fpath = f'{path}/{hash}.{ext}'
    if not Path(fpath).is_file():
        try:
            r = httpx.get(url, timeout=30)
            im = Image.open(io.BytesIO(r.content))
            im.save(fpath)
            return fpath         
        except Exception as e:
            logger.exception(e)
    else:
        if Path(fpath).exists():
            return fpath
        else:
            pass

In [10]:
from tqdm.contrib.concurrent import thread_map

In [11]:
data = list(df.itertuples(index=False))

In [13]:
fpaths = thread_map(download_image, data)

  0%|          | 0/72707 [00:00<?, ?it/s]

2021-05-15 11:29:29.940 | ERROR    | __main__:download_image:13 - cannot identify image file <_io.BytesIO object at 0x7f0077b4d9b0>
Traceback (most recent call last):

  File "/usr/lib/python3.7/threading.py", line 890, in _bootstrap
    self._bootstrap_inner()
    │    └ <function Thread._bootstrap_inner at 0x7f00b1aa4710>
    └ <Thread(ThreadPoolExecutor-1_2, started daemon 139640276670208)>
  File "/usr/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
    │    └ <function Thread.run at 0x7f00b1aa44d0>
    └ <Thread(ThreadPoolExecutor-1_2, started daemon 139640276670208)>
  File "/usr/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
    │    │        │    │        │    └ {}
    │    │        │    │        └ <Thread(ThreadPoolExecutor-1_2, started daemon 139640276670208)>
    │    │        │    └ (<weakref at 0x7f007eac08f0; to 'ThreadPoolExecutor' at 0x7f007e7f3890>, <_queue.SimpleQueue object at 0x7f007eac02f0>, <bo

In [15]:
fpaths;

In [16]:
#df['dl_fpath'] = df.progress_apply(download_image, axis=1)

In [17]:
df['dl_fpath'] = fpaths

In [18]:
df.to_csv(path/'loc-images-labels.csv')

In [10]:
pd.read_csv(path/'loc-images-labels.csv')['dl_fpath'][0]

'/content/drive/MyDrive/IIIF_AI_project/loc-images-labels/7602538306817500955.jpg'

In [12]:
Path('/content/drive/MyDrive/IIIF_AI_project/loc-images-labels/7602538306817500955.jpg').exists()

True